# 45. 나라면 이런거 사겠다(4) 강환국 슈퍼가치전략

[45강의](https://www.youtube.com/watch?v=cnz5N86OyJ8) 를 파이썬으로 구현해보겠습니다.

이 전략은 강환국님의 저서 [할 수 있다! 퀀트 투자](https://ridibooks.com/v2/Detail?id=2596000005&_s=instant&_q=%ED%95%A0%EC%88%98%20%EC%9E%88%EB%8B%A4) 에도 소개된 적이 있는 전략입니다.

## 설치

Pandas 가 있어야 합니다. 설치가 안되어있으시면 다음 셀을 실행시키세요.

In [1]:
import sys
!{sys.executable} -m pip install pandas numpy 2&>1 >/dev/null

## 다운로드

[네이버 카페 퀀트킹(QuantKing)](https://cafe.naver.com/quantking)에서 최신 엑셀 파일을 다운받으세요.

저는 **QuantKing(2019.02.15).xlsb** 파일을 받았습니다. 받으면 확장자 **xlsb** 파일이 보입니다. 엑셀로 여셔서 **QuantKing.xlsx** 파일로 다시 저장해주세요.

## 파일 로딩

파일을 열어서 조사해 보겠습니다. Pandas의 read_excel() 이란 함수를 이용하면 열 수 있네요.

In [2]:
import pandas as pd

In [3]:
filepath = 'QuantKing.xlsx'
df = pd.read_excel(filepath, sheet_name='퀀트데이타', skiprows=2).drop('Unnamed: 0', axis=1)
df.head()

,코드 번호,회사명,업종 (대),업종 (소),코스피코스닥,주가 (원),시가총액 (억),상장주식수 (만주),자사주 (만주),자사주 비중 (%),...,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,19년 1Q(E).2,18년4Q 매출액,18년4Q 영업이익,18년4Q 순이익,18년4Q 매출액.1,18년4Q 영업이익.1,18년4Q 순이익.1
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,46050,2749085,596978.0,0,0.0,...,109815,129674,83301.00,67567.0,592650.00,108006.00,83301.00,630816.0,132917.0,100074.0
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,73800,537266,72800.0,4400,6.0,...,43299,46936,33962.90,15272.0,99380.81,44300.75,33962.90,100565.0,50087.0,41127.0
2,A068270,셀트리온,제약 및 바이오,바이오,코스피,209500,268052,12795.0,85,0.7,...,840,562,530.00,633.0,NaN,NaN,NaN,2514.0,785.0,530.0
3,A051910,LG화학,화학,종합 화학,코스피,380000,268251,7059.0,36,0.5,...,4777,3434,1201.46,3250.0,73427.12,2895.59,1201.46,67891.0,3136.0,1906.0
4,A005380,현대차,자동차 및 관련부품,완성차,코스피,121000,258539,21367.0,939,4.4,...,7006,2692,-1297.74,7997.0,256695.34,5011.27,-1297.74,260649.0,7517.0,6726.0


강의에서 사용하는 컬럼만 잘라서 보관하도록 하겠습니다.

In [4]:
df = df[['회사명',
         '시가총액\n(억)',
         '과거\nPCR',
         '발표\nPSR',
         '발표\nPBR',
         '발표\nPER',
         '유동\n비율\n(%)',
         '단순\n부채비율\n(%)'
        ]].set_index('회사명')
df.head()

,시가총액 (억),과거 PCR,발표 PSR,발표 PBR,발표 PER,유동 비율 (%),단순 부채비율 (%)
회사명,,,,,,,
삼성전자,2749085,4.15,1.13,1.13,6.26,235.968130,39
SK하이닉스,537266,2.68,1.33,1.15,3.46,176.969777,32
셀트리온,268052,60.30,26.44,10.58,79.28,258.906134,31
LG화학,268251,11.23,0.95,1.56,18.22,183.438248,65
현대차,258539,5.10,0.27,0.38,17.14,101.592719,140


## 강환국 슈퍼 가치 전략이란?

- <span style="color:red">**가치와 관련된 4개 지표 통합**</span>: 저 PER + 저 PCR + 저 PBR + 저 PSR
- 그렇다고 아예 가치만 볼 수는 없음. 퀄리티도 좀 보자 (책에서는 가치만 본다고 나옵니다!)
  - 부채비율 < 100%, 유동비율 > 150%
- 책에 보면 연 1회 리발란싱 한다고 하구요

In [5]:
# 가치관련 평균내기

df = df[df['과거\nPCR'] != 0]
df = df[df['발표\nPSR'] != 0]
df = df[df['발표\nPBR'] != 0]
df = df[df['발표\nPER'] != 0]

df['1/PCR'] = 1 / df['과거\nPCR']
df['1/PSR'] = 1 / df['발표\nPSR']
df['1/PBR'] = 1 / df['발표\nPBR']
df['1/PER'] = 1 / df['발표\nPER']

df['순위 PBR'] = df['1/PBR'].rank(ascending=False)
df['순위 PER'] = df['1/PER'].rank(ascending=False)
df['순위 PCR'] = df['1/PCR'].rank(ascending=False)
df['순위 PSR'] = df['1/PSR'].rank(ascending=False)
df['통합순위'] = df[['순위 PBR', '순위 PER', '순위 PCR', '순위 PSR']].mean(axis=1)

# 퀄리티 관련 필터링

df = df[df['유동\n비율\n(%)'] > 150]
df = df[df['단순\n부채비율\n(%)'] < 100]

# 원하는 컬럼만 걸러내기

df = df[['시가총액\n(억)',
         '순위 PCR',
         '순위 PSR',
         '순위 PBR',
         '순위 PER',
         '통합순위',
         '유동\n비율\n(%)',
         '단순\n부채비율\n(%)']]

df.sort_values(by='통합순위').head(n=20)

,시가총액 (억),순위 PCR,순위 PSR,순위 PBR,순위 PER,통합순위,유동 비율 (%),단순 부채비율 (%)
회사명,,,,,,,,
차이나그레이트,386,4.5,23.0,2.0,4.0,8.375,786.709515,13
세아홀딩스,3848,51.0,16.5,10.5,71.0,37.250,157.370091,80
대원산업,1182,90.0,117.5,136.5,70.0,103.500,227.383515,65
한국전자홀딩스,388,74.5,102.0,48.5,312.5,134.375,189.565935,48
에스앤씨엔진그룹,623,29.0,543.0,6.5,38.5,154.250,1196.223610,5
동부건설,1782,251.5,166.0,227.5,22.0,166.750,175.062229,99
수출포장,822,142.0,310.0,80.0,161.0,173.250,192.530464,30
GRT,1381,78.5,518.0,116.0,15.0,181.875,520.966929,20
화성산업,1855,42.5,446.5,214.0,40.0,185.750,316.040000,67


결과가 동영상과 완전 같습니다 (당연히 그래야죠.)

여기서 회사들 하나하나 재무재표 보면서 살짝 걸러냅니다.

그리고, 소형주로만도 보시는데요

In [6]:
# 소형주만 골라보기
(df.sort_values(by='시가총액\n(억)')
   .iloc[:180]
   .sort_values(by='통합순위')
   .head(n=20))

,시가총액 (억),순위 PCR,순위 PSR,순위 PBR,순위 PER,통합순위,유동 비율 (%),단순 부채비율 (%)
회사명,,,,,,,,
차이나그레이트,386,4.5,23.0,2.0,4.0,8.375,786.709515,13
한국전자홀딩스,388,74.5,102.0,48.5,312.5,134.375,189.565935,48
성우전자,554,180.0,378.5,227.5,276.5,265.625,309.821498,26
한일화학,402,413.0,227.5,136.5,523.5,325.125,433.549982,29
경남스틸,545,503.0,166.0,396.5,248.0,328.375,195.190000,74
에이텍티앤,446,98.0,467.5,908.5,54.0,382.000,257.497633,39
제이엠티,563,318.0,435.0,908.5,87.5,437.250,156.732122,89
엑사이엔씨,436,665.0,149.0,518.5,428.0,440.125,152.205082,92
화신정공,467,290.0,196.0,327.0,1089.0,475.500,180.130000,37


## 결론

- 가치로만 주식을 뽑아내기 때문에 우량주를 좋아하시는 강환국님은 그렇게 좋아하시는 전략은 아님 (강의에서처럼 하나하나 재무재표 보고 걸러야 함)
- 강의와 비슷하게 **슈퍼가치 전략만 사용**해서, [2017년 11월 ~ 2018년 12월까지 그 안좋은 장에서 20%나 수익을 보신 분의 후기도 있음](https://www.facebook.com/doerahn/posts/10158101815144554)
- [11강의](11-value-quality-momentum-korea.ipynb)나 [26강의](26-big-mix-korea.ipynb)처럼 가치 + 우량주  + 모멘텀 다 섞는게 그럴듯해 보이긴 하는데...